# Lab 5


Matrix Representation: In this lab you will be creating a simple linear algebra system. In memory, we will represent matrices as nested python lists as we have done in lecture. In the exercises below, you are required to explicitly test every feature you implement, demonstrating it works.

1. Create a `matrix` class with the following properties:
    * It can be initialized in 2 ways:
        1. with arguments `n` and `m`, the size of the matrix. A newly instanciated matrix will contain all zeros.
        2. with a list of lists of values. Note that since we are using lists of lists to implement matrices, it is possible that not all rows have the same number of columns. Test explicitly that the matrix is properly specified.
    * Matrix instances `M` can be indexed with `M[i][j]` and `M[i,j]`.
    * Matrix assignment works in 2 ways:
        1. If `M_1` and `M_2` are `matrix` instances `M_1=M_2` sets the values of `M_1` to those of `M_2`, if they are the same size. Error otherwise.
        2. In example above `M_2` can be a list of lists of correct size.


In [28]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with dimensions
            n, m = args
            if not isinstance(n, int) or not isinstance(m, int):
                raise TypeError("Dimensions must be integers")
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions must be positive")
            self.rows = [[0 for _ in range(m)] for _ in range(n)]
        
        elif len(args) == 1:  # Initialize with list of lists
            if not isinstance(args[0], list) or not all(isinstance(row, list) for row in args[0]):
                raise TypeError("Input must be a list of lists")
            if not args[0]:  # Empty list
                raise ValueError("Matrix cannot be empty")
            
            # Check if all rows have the same length
            row_lengths = [len(row) for row in args[0]]
            if len(set(row_lengths)) != 1:
                raise ValueError("All rows must have the same length")
            if row_lengths[0] == 0:
                raise ValueError("Matrix cannot have zero columns")
            
            # Deep copy the input list to avoid reference issues
            self.rows = [[val for val in row] for row in args[0]]
        
        else:
            raise TypeError("Matrix must be initialized with either dimensions (n,m) or a list of lists")
    
    def __str__(self):
        return "\n".join([" ".join(str(x) for x in row) for row in self.rows])
    
    def __repr__(self):
        return f"Matrix({self.rows})"
    
    def get_dimensions(self):
        return len(self.rows), len(self.rows[0])
    def __getitem__(self, key):
        if isinstance(key, tuple):  # M[i,j] syntax
            if len(key) != 2:
                raise IndexError("Matrix indexing requires 2 indices")
            i, j = key
            return self.rows[i][j]
        else:  # M[i][j] syntax
            return self.rows[key]
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # M[i,j] syntax
            if len(key) != 2:
                raise IndexError("Matrix indexing requires 2 indices")
            i, j = key
            self.rows[i][j] = value
        else:  # M[i][j] syntax
            if isinstance(value, list):
                if len(value) != len(self.rows[0]):
                    raise ValueError("Row assignment must maintain matrix dimensions")
                self.rows[key] = value.copy()
            else:
                raise TypeError("Row assignment requires a list")
    
    def __eq__(self, other):
        """Check if two matrices have the same dimensions and values"""
        if isinstance(other, Matrix):
            return self.rows == other.rows
        return False
    
    def copy(self):
        """Create a deep copy of the matrix"""
        return Matrix(self.rows)

    def __assign_check(self, other):
        """Helper method to check if assignment is valid"""
        if isinstance(other, Matrix):
            other_rows = other.rows
        elif isinstance(other, list) and all(isinstance(row, list) for row in other):
            other_rows = other
        else:
            raise TypeError("Assignment must be from another matrix or list of lists")
        
        # Check dimensions
        if len(other_rows) != len(self.rows) or \
           any(len(row) != len(self.rows[0]) for row in other_rows):
            raise ValueError("Matrices must have the same dimensions for assignment")
        
        return other_rows

    def assign(self, other):
        """Assignment operation"""
        other_rows = self.__assign_check(other)
        # Perform deep copy of values
        self.rows = [[val for val in row] for row in other_rows]

In [29]:
# Test cases
def run_tests():
    print("Running Matrix class tests...\n")
    
    # Test 1: Initialize with dimensions
    print("Test 1: Initialize with dimensions")
    try:
        m1 = Matrix(3, 3)
        print("Success: Created zero matrix:")
        print(m1)
        print(f"Dimensions: {m1.get_dimensions()}\n")
    except Exception as e:
        print(f"Failed: {e}\n")
    
    # Test 2: Initialize with list of lists
    print("Test 2: Initialize with list of lists")
    try:
        m2 = Matrix([[1, 2, 3], [4, 5, 6]])
        print("Success: Created matrix from list:")
        print(m2)
        print(f"Dimensions: {m2.get_dimensions()}\n")
    except Exception as e:
        print(f"Failed: {e}\n")
    
    # Test 3: Invalid dimensions
    print("Test 3: Invalid dimensions")
    try:
        m3 = Matrix(0, 3)
        print("Failed: Should have raised an error")
    except ValueError as e:
        print(f"Success: Caught invalid dimensions - {e}\n")
    
    # Test 4: Uneven rows
    print("Test 4: Uneven rows")
    try:
        m4 = Matrix([[1, 2, 3], [4, 5]])
        print("Failed: Should have raised an error")
    except ValueError as e:
        print(f"Success: Caught uneven rows - {e}\n")
    
    # Test 5: Empty matrix
    print("Test 5: Empty matrix")
    try:
        m5 = Matrix([])
        print("Failed: Should have raised an error")
    except ValueError as e:
        print(f"Success: Caught empty matrix - {e}\n")
    
    # Test 6: Invalid input type
    print("Test 6: Invalid input type")
    try:
        m6 = Matrix("not a matrix")
        print("Failed: Should have raised an error")
    except TypeError as e:
        print(f"Success: Caught invalid input type - {e}\n")
    # Test 7: Basic indexing
    print("Test 7: Basic indexing")
    try:
        m1 = Matrix([[1, 2, 3], [4, 5, 6]])
        print("Testing M[i][j] syntax:")
        print(f"m1[0][1] = {m1[0][1]}")  # Should be 2
        print("Testing M[i,j] syntax:")
        print(f"m1[1,2] = {m1[1,2]}")    # Should be 6
        print()
    except Exception as e:
        print(f"Failed: {e}\n")
    
    # Test 8: Index assignment
    print("Test 8: Index assignment")
    try:
        m1 = Matrix([[1, 2, 3], [4, 5, 6]])
        m1[0,1] = 10
        m1[1][2] = 20
        print("After assignments m1[0,1] = 10 and m1[1][2] = 20:")
        print(m1)
        print()
    except Exception as e:
        print(f"Failed: {e}\n")
    
    # Test 9: Matrix assignment from another matrix
    print("Test 9: Matrix assignment from another matrix")
    try:
        m1 = Matrix([[1, 2, 3], [4, 5, 6]])
        m2 = Matrix([[7, 8, 9], [10, 11, 12]])
        m1.assign(m2)
        print("After m1.assign(m2):")
        print(m1)
        print()
    except Exception as e:
        print(f"Failed: {e}\n")
    
    # Test 10: Matrix assignment from list of lists
    print("Test 10: Matrix assignment from list of lists")
    try:
        m1 = Matrix([[1, 2, 3], [4, 5, 6]])
        m1.assign([[13, 14, 15], [16, 17, 18]])
        print("After assigning from list of lists:")
        print(m1)
        print()
    except Exception as e:
        print(f"Failed: {e}\n")
    
    # Test 11: Invalid assignments
    print("Test 11: Invalid assignments")
    try:
        m1 = Matrix([[1, 2, 3], [4, 5, 6]])
        m2 = Matrix([[1, 2], [3, 4]])  # Wrong dimensions
        m1.assign(m2)
        print("Failed: Should have raised an error")
    except ValueError as e:
        print(f"Success: Caught dimension mismatch - {e}")
    
    try:
        m1.assign([[1, 2], [3, 4]])  # Wrong dimensions
        print("Failed: Should have raised an error")
    except ValueError as e:
        print(f"Success: Caught dimension mismatch - {e}")
    
    try:
        m1.assign("not a matrix")
        print("Failed: Should have raised an error")
    except TypeError as e:
        print(f"Success: Caught invalid type - {e}\n")

if __name__ == "__main__":
    run_tests()

Running Matrix class tests...

Test 1: Initialize with dimensions
Success: Created zero matrix:
0 0 0
0 0 0
0 0 0
Dimensions: (3, 3)

Test 2: Initialize with list of lists
Success: Created matrix from list:
1 2 3
4 5 6
Dimensions: (2, 3)

Test 3: Invalid dimensions
Success: Caught invalid dimensions - Dimensions must be positive

Test 4: Uneven rows
Success: Caught uneven rows - All rows must have the same length

Test 5: Empty matrix
Success: Caught empty matrix - Matrix cannot be empty

Test 6: Invalid input type
Success: Caught invalid input type - Input must be a list of lists

Test 7: Basic indexing
Testing M[i][j] syntax:
m1[0][1] = 2
Testing M[i,j] syntax:
m1[1,2] = 6

Test 8: Index assignment
After assignments m1[0,1] = 10 and m1[1][2] = 20:
1 10 3
4 5 20

Test 9: Matrix assignment from another matrix
After m1.assign(m2):
7 8 9
10 11 12

Test 10: Matrix assignment from list of lists
After assigning from list of lists:
13 14 15
16 17 18

Test 11: Invalid assignments
Success: Cau

2. Add the following methods:
    * `shape()`: returns a tuple `(n,m)` of the shape of the matrix.
    * `transpose()`: returns a new matrix instance which is the transpose of the matrix.
    * `row(n)` and `column(n)`: that return the nth row or column of the matrix M as a new appropriately shaped matrix object.
    * `to_list()`: which returns the matrix as a list of lists.
    *  `block(n_0,n_1,m_0,m_1)` that returns a smaller matrix located at the n_0 to n_1 columns and m_0 to m_1 rows. 
    * (Extra credit) Modify `__getitem__` implemented above to support slicing.
        

In [52]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with dimensions
            n, m = args
            if not isinstance(n, int) or not isinstance(m, int):
                raise TypeError("Dimensions must be integers")
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions must be positive")
            self.rows = [[0 for _ in range(m)] for _ in range(n)]
        
        elif len(args) == 1:  # Initialize with list of lists
            if not isinstance(args[0], list) or not all(isinstance(row, list) for row in args[0]):
                raise TypeError("Input must be a list of lists")
            if not args[0]:  # Empty list
                raise ValueError("Matrix cannot be empty")
            
            # Check if all rows have the same length
            row_lengths = [len(row) for row in args[0]]
            if len(set(row_lengths)) != 1:
                raise ValueError("All rows must have the same length")
            if row_lengths[0] == 0:
                raise ValueError("Matrix cannot have zero columns")
            
            # Deep copy the input list to avoid reference issues
            self.rows = [[val for val in row] for row in args[0]]
        
        else:
            raise TypeError("Matrix must be initialized with either dimensions (n,m) or a list of lists")
    
    def __str__(self):
        return "\n".join([" ".join(str(x) for x in row) for row in self.rows])
    
    def __repr__(self):
        return f"Matrix({self.rows})"
    
    def get_dimensions(self):
        return len(self.rows), len(self.rows[0])
    def __getitem__(self, key):
        if isinstance(key, tuple):  # M[i,j] syntax
            if len(key) != 2:
                raise IndexError("Matrix indexing requires 2 indices")
            i, j = key
            return self.rows[i][j]
        else:  # M[i][j] syntax
            return self.rows[key]
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # M[i,j] syntax
            if len(key) != 2:
                raise IndexError("Matrix indexing requires 2 indices")
            i, j = key
            self.rows[i][j] = value
        else:  # M[i][j] syntax
            if isinstance(value, list):
                if len(value) != len(self.rows[0]):
                    raise ValueError("Row assignment must maintain matrix dimensions")
                self.rows[key] = value.copy()
            else:
                raise TypeError("Row assignment requires a list")
    
    def __eq__(self, other):
        """Check if two matrices have the same dimensions and values"""
        if isinstance(other, Matrix):
            return self.rows == other.rows
        return False
    
    def copy(self):
        """Create a deep copy of the matrix"""
        return Matrix(self.rows)

    def __assign_check(self, other):
        """checking if assignment is valid"""
        if isinstance(other, Matrix):
            other_rows = other.rows
        elif isinstance(other, list) and all(isinstance(row, list) for row in other):
            other_rows = other
        else:
            raise TypeError("Assignment must be from another matrix or list of lists")
        
        # Checking dimensions
        if len(other_rows) != len(self.rows) or \
           any(len(row) != len(self.rows[0]) for row in other_rows):
            raise ValueError("Matrices must have the same dimensions for assignment")
        
        return other_rows

    def assign(self, other):
        """Assignment operation"""
        other_rows = self.__assign_check(other)
        # Perform deep copy of values
        self.rows = [[val for val in row] for row in other_rows]

    def shape(self):
        """Returns the shape of the matrix as tuple (n,m)"""
        return len(self.rows), len(self.rows[0])

    def transpose(self):
        """Returns a new matrix which is the transpose of the current matrix"""
        n, m = self.shape()
        return Matrix([[self.rows[j][i] for j in range(n)] for i in range(m)])

    def row(self, n):
        """Returns the nth row as a new 1xm matrix"""
        if not 0 <= n < len(self.rows):
            raise IndexError("Row index out of range")
        return Matrix([self.rows[n][:]])

    def column(self, n):
        """Returns the nth column as a new nx1 matrix"""
        if not 0 <= n < len(self.rows[0]):
            raise IndexError("Column index out of range")
        return Matrix([[row[n]] for row in self.rows])

    def to_list(self):
        """Returns the matrix as a list of lists"""
        return [[val for val in row] for row in self.rows]

    def block(self, n_0, n_1, m_0, m_1):
        """Returns a submatrix located at n_0 to n_1 columns and m_0 to m_1 rows.
        """
        if not (0 <= m_0 <= m_1 < len(self.rows[0])):
            raise IndexError("Column indices out of range")
        if not (0 <= n_0 <= n_1 < len(self.rows)):
            raise IndexError("Row indices out of range")
        
        return Matrix([row[m_0:m_1+1] for row in self.rows[n_0:n_1+1]])

In [36]:
def run_tests():
    print("Running Matrix class tests...\n")
    
    # Create a test matrix
    m = Matrix([[1, 2, 3, 4],
                [5, 6, 7, 8],
                [9, 10, 11, 12]])
    print("Test matrix:")
    print(m)
    print()
    
    # Test 1: shape()
    print("Test 1: shape()")
    shape = m.shape()
    print(f"Matrix shape: {shape}")
    print()
    
    # Test 2: transpose()
    print("Test 2: transpose()")
    m_t = m.transpose()
    print("Transposed matrix:")
    print(m_t)
    print(f"Transposed shape: {m_t.shape()}")
    print()
    
    # Test 3: row()
    print("Test 3: row()")
    row_1 = m.row(1)
    print("Second row (index 1):")
    print(row_1)
    print(f"Row shape: {row_1.shape()}")
    print()
    
    # Test 4: column()
    print("Test 4: column()")
    col_1 = m.column(1)
    print("Second column (index 1):")
    print(col_1)
    print(f"Column shape: {col_1.shape()}")
    print()
    
    # Test 5: to_list()
    print("Test 5: to_list()")
    lst = m.to_list()
    print("Matrix as list:")
    print(lst)
    print()
    
    # Test 6: block()
    block = m.block(0, 1, 1, 2)  # Should get rows 0-1, columns 1-2
    print("Block from rows 0-1 and columns 1-2:")
    print(block)
    print(f"Block shape: {block.shape()}")
    print()
    
    # Test error cases
    print("Test 7: Error cases")
    try:
        m.row(5)  # Invalid row index
        print("Failed: Should raise error for invalid row index")
    except IndexError as e:
        print(f"Success: {e}")
    
    try:
        m.column(5)  # Invalid column index
        print("Failed: Should raise error for invalid column index")
    except IndexError as e:
        print(f"Success: {e}")
    
    try:
        m.block(0, 5, 0, 1)  # Invalid block indices
        print("Failed: Should raise error for invalid block indices")
    except IndexError as e:
        print(f"Success: {e}")
run_tests()

Running Matrix class tests...

Test matrix:
1 2 3 4
5 6 7 8
9 10 11 12

Test 1: shape()
Matrix shape: (3, 4)

Test 2: transpose()
Transposed matrix:
1 5 9
2 6 10
3 7 11
4 8 12
Transposed shape: (4, 3)

Test 3: row()
Second row (index 1):
5 6 7 8
Row shape: (1, 4)

Test 4: column()
Second column (index 1):
2
6
10
Column shape: (3, 1)

Test 5: to_list()
Matrix as list:
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]

Block from rows 0-1 and columns 1-2:
2 3
6 7
Block shape: (2, 2)

Test 7: Error cases
Success: Row index out of range
Success: Column index out of range
Success: Column indices out of range


3. Write functions that create special matrices (note these are standalone functions, not member functions of your `matrix` class):
    * `constant(n,m,c)`: returns a `n` by `m` matrix filled with floats of value `c`.
    * `zeros(n,m)` and `ones(n,m)`: return `n` by `m` matrices filled with floats of value `0` and `1`, respectively.
    * `eye(n)`: returns the n by n identity matrix.

In [37]:
def constant(n, m, c):
    """Returns an n by m matrix filled with floats of value c"""
    if not isinstance(n, int) or not isinstance(m, int):
        raise TypeError("Dimensions must be integers")
    if n <= 0 or m <= 0:
        raise ValueError("Dimensions must be positive")
    return Matrix([[float(c) for _ in range(m)] for _ in range(n)])

def zeros(n, m):
    """Returns an n by m matrix filled with zeros"""
    return constant(n, m, 0)

def ones(n, m):
    """Returns an n by m matrix filled with ones"""
    return constant(n, m, 1)

def eye(n):
    """Returns an n by n identity matrix"""
    if not isinstance(n, int):
        raise TypeError("Dimension must be an integer")
    if n <= 0:
        raise ValueError("Dimension must be positive")
    result = zeros(n, n)
    for i in range(n):
        result[i,i] = 1.0
    return result


In [ ]:
def run_tests():
    print("Testing special matrix creation functions...\n")
    
    # Test constant matrix
    print("Test 1: constant(2, 3, 7.5)")
    m1 = constant(2, 3, 7.5)
    print(m1)
    print(f"Shape: {m1.shape()}")
    print()
    
    # Test zeros matrix
    print("Test 2: zeros(2, 3)")
    m2 = zeros(2, 3)
    print(m2)
    print(f"Shape: {m2.shape()}")
    print()
    
    # Test ones matrix
    print("Test 3: ones(3, 2)")
    m3 = ones(3, 2)
    print(m3)
    print(f"Shape: {m3.shape()}")
    print()
    
    # Test identity matrix
    print("Test 4: eye(3)")
    m4 = eye(3)
    print(m4)
    print(f"Shape: {m4.shape()}")
    print()
    
    # Test error cases
    print("Test 5: Error cases")
    try:
        constant(0, 3, 1)
        print("Failed: Should raise error for zero dimension")
    except ValueError as e:
        print(f"Success: {e}")
    
    try:
        constant("2", 3, 1)
        print("Failed: Should raise error for non-integer dimension")
    except TypeError as e:
        print(f"Success: {e}")
    
    try:
        eye(0)
        print("Failed: Should raise error for zero dimension")
    except ValueError as e:
        print(f"Success: {e}")

In [38]:
run_tests()

Running Matrix class tests...

Test matrix:
1 2 3 4
5 6 7 8
9 10 11 12

Test 1: shape()
Matrix shape: (3, 4)

Test 2: transpose()
Transposed matrix:
1 5 9
2 6 10
3 7 11
4 8 12
Transposed shape: (4, 3)

Test 3: row()
Second row (index 1):
5 6 7 8
Row shape: (1, 4)

Test 4: column()
Second column (index 1):
2
6
10
Column shape: (3, 1)

Test 5: to_list()
Matrix as list:
[[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]]

Block from rows 0-1 and columns 1-2:
2 3
6 7
Block shape: (2, 2)

Test 7: Error cases
Success: Row index out of range
Success: Column index out of range
Success: Column indices out of range


4. Add the following member functions to your class. Make sure to appropriately test the dimensions of the matrices to make sure the operations are correct.
    * `M.scalarmul(c)`: a matrix that is scalar product $cM$, where every element of $M$ is multiplied by $c$.
    * `M.add(N)`: adds two matrices $M$ and $N$. Don’t forget to test that the sizes of the matrices are compatible for this and all other operations.
    * `M.sub(N)`: subtracts two matrices $M$ and $N$.
    * `M.mat_mult(N)`: returns a matrix that is the matrix product of two matrices $M$ and $N$.
    * `M.element_mult(N)`: returns a matrix that is the element-wise product of two matrices $M$ and $N$.
    * `M.equals(N)`: returns true/false if $M==N$.

In [45]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with dimensions
            n, m = args
            if not isinstance(n, int) or not isinstance(m, int):
                raise TypeError("Dimensions must be integers")
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions must be positive")
            self.rows = [[0 for _ in range(m)] for _ in range(n)]
        
        elif len(args) == 1:  # Initialize with list of lists
            if not isinstance(args[0], list) or not all(isinstance(row, list) for row in args[0]):
                raise TypeError("Input must be a list of lists")
            if not args[0]:  # Empty list
                raise ValueError("Matrix cannot be empty")
            
            # Check if all rows have the same length
            row_lengths = [len(row) for row in args[0]]
            if len(set(row_lengths)) != 1:
                raise ValueError("All rows must have the same length")
            if row_lengths[0] == 0:
                raise ValueError("Matrix cannot have zero columns")
            
            # Deep copy the input list to avoid reference issues
            self.rows = [[val for val in row] for row in args[0]]
        
        else:
            raise TypeError("Matrix must be initialized with either dimensions (n,m) or a list of lists")
    
    def __str__(self):
        return "\n".join([" ".join(str(x) for x in row) for row in self.rows])
    
    def __repr__(self):
        return f"Matrix({self.rows})"
    
    def get_dimensions(self):
        return len(self.rows), len(self.rows[0])
    def __getitem__(self, key):
        if isinstance(key, tuple):  # M[i,j] syntax
            if len(key) != 2:
                raise IndexError("Matrix indexing requires 2 indices")
            i, j = key
            return self.rows[i][j]
        else:  # M[i][j] syntax
            return self.rows[key]
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # M[i,j] syntax
            if len(key) != 2:
                raise IndexError("Matrix indexing requires 2 indices")
            i, j = key
            self.rows[i][j] = value
        else:  # M[i][j] syntax
            if isinstance(value, list):
                if len(value) != len(self.rows[0]):
                    raise ValueError("Row assignment must maintain matrix dimensions")
                self.rows[key] = value.copy()
            else:
                raise TypeError("Row assignment requires a list")
    
    def __eq__(self, other):
        """Check if two matrices have the same dimensions and values"""
        if isinstance(other, Matrix):
            return self.rows == other.rows
        return False
    
    def copy(self):
        """Create a deep copy of the matrix"""
        return Matrix(self.rows)

    def __assign_check(self, other):
        """Helper method to check if assignment is valid"""
        if isinstance(other, Matrix):
            other_rows = other.rows
        elif isinstance(other, list) and all(isinstance(row, list) for row in other):
            other_rows = other
        else:
            raise TypeError("Assignment must be from another matrix or list of lists")
        
        # Check dimensions
        if len(other_rows) != len(self.rows) or \
           any(len(row) != len(self.rows[0]) for row in other_rows):
            raise ValueError("Matrices must have the same dimensions for assignment")
        
        return other_rows

    def assign(self, other):
        """Assignment operation"""
        other_rows = self.__assign_check(other)
        # Perform deep copy of values
        self.rows = [[val for val in row] for row in other_rows]

    def shape(self):
        """Returns the shape of the matrix as tuple (n,m)"""
        return len(self.rows), len(self.rows[0])

    def transpose(self):
        """Returns a new matrix which is the transpose of the current matrix"""
        n, m = self.shape()
        return Matrix([[self.rows[j][i] for j in range(n)] for i in range(m)])

    def row(self, n):
        """Returns the nth row as a new 1xm matrix"""
        if not 0 <= n < len(self.rows):
            raise IndexError("Row index out of range")
        return Matrix([self.rows[n][:]])

    def column(self, n):
        """Returns the nth column as a new nx1 matrix"""
        if not 0 <= n < len(self.rows[0]):
            raise IndexError("Column index out of range")
        return Matrix([[row[n]] for row in self.rows])

    def to_list(self):
        """Returns the matrix as a list of lists"""
        return [[val for val in row] for row in self.rows]

    def block(self, n_0, n_1, m_0, m_1):
        """Returns a submatrix located at n_0 to n_1 columns and m_0 to m_1 rows.
        Parameters:
        n_0, n_1: starting and ending row indices
        m_0, m_1: starting and ending column indices
        """
        if not (0 <= m_0 <= m_1 < len(self.rows[0])):
            raise IndexError("Column indices out of range")
        if not (0 <= n_0 <= n_1 < len(self.rows)):
            raise IndexError("Row indices out of range")
        
        return Matrix([row[m_0:m_1+1] for row in self.rows[n_0:n_1+1]])
    
    def scalarmul(self, c):
        """Returns a new matrix that is the scalar product cM"""
        return Matrix([[c * val for val in row] for row in self.rows])

    def add(self, other):
        """Returns a new matrix that is the sum of this matrix and other"""
        if not isinstance(other, Matrix):
            raise TypeError("Addition is only defined between matrices")
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match for addition")
        return Matrix([[self.rows[i][j] + other.rows[i][j] 
                       for j in range(len(self.rows[0]))]
                       for i in range(len(self.rows))])

    def sub(self, other):
        """Returns a new matrix that is the difference of this matrix and other"""
        if not isinstance(other, Matrix):
            raise TypeError("Subtraction is only defined between matrices")
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match for subtraction")
        return Matrix([[self.rows[i][j] - other.rows[i][j] 
                       for j in range(len(self.rows[0]))]
                       for i in range(len(self.rows))])

    def mat_mult(self, other):
        """Returns a new matrix that is the matrix product of this matrix and other"""
        if not isinstance(other, Matrix):
            raise TypeError("Matrix multiplication is only defined between matrices")
        m_rows, m_cols = self.shape()
        n_rows, n_cols = other.shape()
        if m_cols != n_rows:
            raise ValueError(f"Matrix dimensions incompatible for multiplication: {self.shape()} and {other.shape()}")
        
        result = [[sum(self.rows[i][k] * other.rows[k][j] 
                      for k in range(m_cols))
                  for j in range(n_cols)]
                  for i in range(m_rows)]
        return Matrix(result)

    def element_mult(self, other):
        """Returns a new matrix that is the element-wise product of this matrix and other"""
        if not isinstance(other, Matrix):
            raise TypeError("Element-wise multiplication is only defined between matrices")
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match for element-wise multiplication")
        return Matrix([[self.rows[i][j] * other.rows[i][j] 
                       for j in range(len(self.rows[0]))]
                       for i in range(len(self.rows))])

    def equals(self, other):
        """Returns True if this matrix equals other, False otherwise"""
        if not isinstance(other, Matrix):
            return False
        return self.rows == other.rows

In [46]:
def run_tests():
    print("Testing matrix operations...\n")
    
    # Create test matrices
    A = Matrix([[1, 2], [3, 4]])
    B = Matrix([[5, 6], [7, 8]])
    C = Matrix([[1, 2, 3], [4, 5, 6]])  # 2x3 matrix for testing dimension mismatch
    
    print("Matrix A:")
    print(A)
    print("\nMatrix B:")
    print(B)
    print("\nMatrix C:")
    print(C)
    print()
    
    # Test scalar multiplication
    print("Test 1: Scalar multiplication (A * 2)")
    print(A.scalarmul(2))
    print()
    
    # Test matrix addition
    print("Test 2: Matrix addition (A + B)")
    print(A.add(B))
    print()
    
    # Test matrix subtraction
    print("Test 3: Matrix subtraction (A - B)")
    print(A.sub(B))
    print()
    
    # Test matrix multiplication
    print("Test 4: Matrix multiplication (A * B)")
    print(A.mat_mult(B))
    print()
    
    # Test element-wise multiplication
    print("Test 5: Element-wise multiplication (A  B)")
    print(A.element_mult(B))
    print()
    
    # Test equality
    print("Test 6: Matrix equality")
    D = Matrix([[1, 2], [3, 4]])
    print(f"A equals A: {A.equals(A)}")  # Should be True
    print(f"A equals B: {A.equals(B)}")  # Should be False
    print(f"A equals D: {A.equals(D)}")  # Should be True
    print()
    
    # Test error cases
    print("Test 7: Error cases")
    try:
        A.add(C)
        print("Failed: Should raise error for dimension mismatch in addition")
    except ValueError as e:
        print(f"Success: {e}")
    
    try:
        A.mat_mult(C)
        print("Failed: Should raise error for dimension mismatch in multiplication")
    except ValueError as e:
        print(f"Success: {e}")
    
    try:
        A.element_mult("not a matrix")
        print("Failed: Should raise error for invalid type")
    except TypeError as e:
        print(f"Success: {e}")


run_tests()

Testing matrix operations...

Matrix A:
1 2
3 4

Matrix B:
5 6
7 8

Matrix C:
1 2 3
4 5 6

Test 1: Scalar multiplication (A * 2)
2 4
6 8

Test 2: Matrix addition (A + B)
6 8
10 12

Test 3: Matrix subtraction (A - B)
-4 -4
-4 -4

Test 4: Matrix multiplication (A * B)
19 22
43 50

Test 5: Element-wise multiplication (A ⊙ B)
5 12
21 32

Test 6: Matrix equality
A equals A: True
A equals B: False
A equals D: True

Test 7: Error cases
Success: Matrix dimensions must match for addition
Failed: Should raise error for dimension mismatch in multiplication
Success: Element-wise multiplication is only defined between matrices


5. Overload python operators to appropriately use your functions in 4 and allow expressions like:
    * 2*M
    * M*2
    * M+N
    * M-N
    * M*N
    * M==N
    * M=N


In [47]:
class Matrix:
  
    
    def __init__(self, *args):
        """
        Initialize a Matrix object in one of two ways:
        1. Matrix(n, m) - Create an n x m zero matrix
        2. Matrix(list_of_lists) - Create a matrix from a 2D list
        
        """
        if len(args) == 2:  # Initialize with dimensions
            n, m = args
            if not isinstance(n, int) or not isinstance(m, int):
                raise TypeError("Dimensions must be integers")
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions must be positive")
            self.rows = [[0 for _ in range(m)] for _ in range(n)]
        
        elif len(args) == 1:  # Initialize with list of lists
            if not isinstance(args[0], list) or not all(isinstance(row, list) for row in args[0]):
                raise TypeError("Input must be a list of lists")
            if not args[0]:  # Empty list
                raise ValueError("Matrix cannot be empty")
            
            # Check if all rows have the same length
            row_lengths = [len(row) for row in args[0]]
            if len(set(row_lengths)) != 1:
                raise ValueError("All rows must have the same length")
            if row_lengths[0] == 0:
                raise ValueError("Matrix cannot have zero columns")
            
            # Deep copy the input list to avoid reference issues
            self.rows = [[val for val in row] for row in args[0]]
        else:
            raise TypeError("Matrix must be initialized with either dimensions (n,m) or a list of lists")

    def scalarmul(self, c):
        """
        Multiply every element in the matrix by a scalar value.
        
        """
        return Matrix([[c * val for val in row] for row in self.rows])

    def add(self, other):
        """
        Add this matrix to another matrix element-wise.
        
        """
        if not isinstance(other, Matrix):
            raise TypeError("Addition is only defined between matrices")
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match for addition")
        return Matrix([[self.rows[i][j] + other.rows[i][j] 
                       for j in range(len(self.rows[0]))]
                       for i in range(len(self.rows))])

    def sub(self, other):
        """
        Subtract another matrix from this matrix element-wise.
        
        """
        if not isinstance(other, Matrix):
            raise TypeError("Subtraction is only defined between matrices")
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match for subtraction")
        return Matrix([[self.rows[i][j] - other.rows[i][j] 
                       for j in range(len(self.rows[0]))]
                       for i in range(len(self.rows))])

    def mat_mult(self, other):
        """
        Perform matrix multiplication with another matrix.
        
        """
        if not isinstance(other, Matrix):
            raise TypeError("Matrix multiplication is only defined between matrices")
        m_rows, m_cols = self.shape()
        n_rows, n_cols = other.shape()
        if m_cols != n_rows:
            raise ValueError(f"Matrix dimensions incompatible for multiplication: {self.shape()} and {other.shape()}")
        
        result = [[sum(self.rows[i][k] * other.rows[k][j] 
                      for k in range(m_cols))
                  for j in range(n_cols)]
                  for i in range(m_rows)]
        return Matrix(result)

    # Operator overloading methods
    def __mul__(self, other):
        """
        Implements multiplication operator (*).
        Supports both matrix multiplication (M * N) and scalar multiplication (M * c).
        
        """
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        try:
            return self.scalarmul(float(other))
        except (TypeError, ValueError):
            raise TypeError("Matrix can only be multiplied by another matrix or a number")

    def __rmul__(self, other):
        """
        Implements reverse multiplication operator (c * M).
        
        """
        try:
            return self.scalarmul(float(other))
        except (TypeError, ValueError):
            raise TypeError("Matrix can only be multiplied by a number from the left")

    def __add__(self, other):
        """
        Implements addition operator (+).
        
        """
        return self.add(other)

    def __sub__(self, other):
        """
        Implements subtraction operator (-).
        
        """
        return self.sub(other)

    def __eq__(self, other):
        """
        Implements equality operator (==).
        Matrices are equal if they have the same dimensions and elements.
    
        """
        if not isinstance(other, Matrix):
            return False
        return self.rows == other.rows

    def shape(self):
        """
        Get the dimensions of the matrix.
        
        """
        return len(self.rows), len(self.rows[0])

    def __str__(self):
        """
        Convert matrix to string representation for printing.
        
        """
        return "\n".join([" ".join(str(x) for x in row) for row in self.rows])


In [48]:
def run_tests():
    
 
    print("Testing matrix operators...\n")
    
    # Create test matrices
    A = Matrix([[1, 2], [3, 4]])
    B = Matrix([[5, 6], [7, 8]])
    C = Matrix([[1, 2, 3], [4, 5, 6]])  # 2x3 matrix for testing dimension mismatch
    
    print("Matrix A:")
    print(A)
    print("\nMatrix B:")
    print(B)
    print("\nMatrix C:")
    print(C)
    print()
    
    # Test scalar multiplication (both sides)
    print("Test 1: Scalar multiplication")
    print("2 * A:")
    print(2 * A)
    print("\nA * 2:")
    print(A * 2)
    print()
    
    # Test matrix addition
    print("Test 2: Matrix addition (A + B)")
    print(A + B)
    print()
    
    # Test matrix subtraction
    print("Test 3: Matrix subtraction (A - B)")
    print(A - B)
    print()
    
    # Test matrix multiplication
    print("Test 4: Matrix multiplication (A * B)")
    print(A * B)
    print()
    
    # Test equality
    print("Test 5: Matrix equality")
    D = Matrix([[1, 2], [3, 4]])
    print(f"A == A: {A == A}")  # Should be True
    print(f"A == B: {A == B}")  # Should be False
    print(f"A == D: {A == D}")  # Should be True
    print()
    
    # Test error cases
    print("Test 6: Error cases")
    try:
        result = A + C
        print("Failed: Should raise error for dimension mismatch in addition")
    except ValueError as e:
        print(f"Success: {e}")
    
    try:
        result = A * C
        print("Failed: Should raise error for dimension mismatch in multiplication")
    except ValueError as e:
        print(f"Success: {e}")
    
    try:
        result = A * "not a number"
        print("Failed: Should raise error for invalid scalar type")
    except TypeError as e:
        print(f"Success: {e}")

run_tests()

Testing matrix operators...

Matrix A:
1 2
3 4

Matrix B:
5 6
7 8

Matrix C:
1 2 3
4 5 6

Test 1: Scalar multiplication
2 * A:
2.0 4.0
6.0 8.0

A * 2:
2.0 4.0
6.0 8.0

Test 2: Matrix addition (A + B)
6 8
10 12

Test 3: Matrix subtraction (A - B)
-4 -4
-4 -4

Test 4: Matrix multiplication (A * B)
19 22
43 50

Test 5: Matrix equality
A == A: True
A == B: False
A == D: True

Test 6: Error cases
Success: Matrix dimensions must match for addition
Failed: Should raise error for dimension mismatch in multiplication
Success: Matrix can only be multiplied by another matrix or a number


6. Demonstrate the basic properties of matrices with your matrix class by creating two 2 by 2 example matrices using your Matrix class and illustrating the following:

$$
(AB)C=A(BC)
$$
$$
A(B+C)=AB+AC
$$
$$
AB\neq BA
$$
$$
AI=A
$$

In [61]:
class Matrix:
    def __init__(self, *args):
        if len(args) == 2:  # Initialize with dimensions
            n, m = args
            if not isinstance(n, int) or not isinstance(m, int):
                raise TypeError("Dimensions must be integers")
            if n <= 0 or m <= 0:
                raise ValueError("Dimensions must be positive")
            self.rows = [[0 for _ in range(m)] for _ in range(n)]
        
        elif len(args) == 1:  # Initialize with list of lists
            if not isinstance(args[0], list) or not all(isinstance(row, list) for row in args[0]):
                raise TypeError("Input must be a list of lists")
            if not args[0]:  # Empty list
                raise ValueError("Matrix cannot be empty")
            
            # Check if all rows have the same length
            row_lengths = [len(row) for row in args[0]]
            if len(set(row_lengths)) != 1:
                raise ValueError("All rows must have the same length")
            if row_lengths[0] == 0:
                raise ValueError("Matrix cannot have zero columns")
            
            # Deep copy the input list to avoid reference issues
            self.rows = [[val for val in row] for row in args[0]]
        
        else:
            raise TypeError("Matrix must be initialized with either dimensions (n,m) or a list of lists")
    
    def __str__(self):
        return "\n".join([" ".join(str(x) for x in row) for row in self.rows])
    
    def __repr__(self):
        return f"Matrix({self.rows})"
    
    def get_dimensions(self):
        return len(self.rows), len(self.rows[0])
    def __getitem__(self, key):
        if isinstance(key, tuple):  # M[i,j] syntax
            if len(key) != 2:
                raise IndexError("Matrix indexing requires 2 indices")
            i, j = key
            return self.rows[i][j]
        else:  # M[i][j] syntax
            return self.rows[key]
    
    def __setitem__(self, key, value):
        if isinstance(key, tuple):  # M[i,j] syntax
            if len(key) != 2:
                raise IndexError("Matrix indexing requires 2 indices")
            i, j = key
            self.rows[i][j] = value
        else:  # M[i][j] syntax
            if isinstance(value, list):
                if len(value) != len(self.rows[0]):
                    raise ValueError("Row assignment must maintain matrix dimensions")
                self.rows[key] = value.copy()
            else:
                raise TypeError("Row assignment requires a list")
    
    def __eq__(self, other):
        """Check if two matrices have the same dimensions and values"""
        if isinstance(other, Matrix):
            return self.rows == other.rows
        return False
    
    def copy(self):
        """Create a deep copy of the matrix"""
        return Matrix(self.rows)

    def __assign_check(self, other):
        """Helper method to check if assignment is valid"""
        if isinstance(other, Matrix):
            other_rows = other.rows
        elif isinstance(other, list) and all(isinstance(row, list) for row in other):
            other_rows = other
        else:
            raise TypeError("Assignment must be from another matrix or list of lists")
        
        # Check dimensions
        if len(other_rows) != len(self.rows) or \
           any(len(row) != len(self.rows[0]) for row in other_rows):
            raise ValueError("Matrices must have the same dimensions for assignment")
        
        return other_rows

    def assign(self, other):
        """Assignment operation"""
        other_rows = self.__assign_check(other)
        # Perform deep copy of values
        self.rows = [[val for val in row] for row in other_rows]

    def shape(self):
        """Returns the shape of the matrix as tuple (n,m)"""
        return len(self.rows), len(self.rows[0])

    def transpose(self):
        """Returns a new matrix which is the transpose of the current matrix"""
        n, m = self.shape()
        return Matrix([[self.rows[j][i] for j in range(n)] for i in range(m)])

    def row(self, n):
        """Returns the nth row as a new 1xm matrix"""
        if not 0 <= n < len(self.rows):
            raise IndexError("Row index out of range")
        return Matrix([self.rows[n][:]])

    def column(self, n):
        """Returns the nth column as a new nx1 matrix"""
        if not 0 <= n < len(self.rows[0]):
            raise IndexError("Column index out of range")
        return Matrix([[row[n]] for row in self.rows])

    def to_list(self):
        """Returns the matrix as a list of lists"""
        return [[val for val in row] for row in self.rows]

    def block(self, n_0, n_1, m_0, m_1):
        """Returns a submatrix located at n_0 to n_1 columns and m_0 to m_1 rows.
        """
        if not (0 <= m_0 <= m_1 < len(self.rows[0])):
            raise IndexError("Column indices out of range")
        if not (0 <= n_0 <= n_1 < len(self.rows)):
            raise IndexError("Row indices out of range")
        
        return Matrix([row[m_0:m_1+1] for row in self.rows[n_0:n_1+1]])
    
    def scalarmul(self, c):
        """Returns a new matrix that is the scalar product cM"""
        return Matrix([[c * val for val in row] for row in self.rows])

    def add(self, other):
        """Returns a new matrix that is the sum of this matrix and other"""
        if not isinstance(other, Matrix):
            raise TypeError("Addition is only defined between matrices")
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match for addition")
        return Matrix([[self.rows[i][j] + other.rows[i][j] 
                       for j in range(len(self.rows[0]))]
                       for i in range(len(self.rows))])

    def sub(self, other):
        """Returns a new matrix that is the difference of this matrix and other"""
        if not isinstance(other, Matrix):
            raise TypeError("Subtraction is only defined between matrices")
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match for subtraction")
        return Matrix([[self.rows[i][j] - other.rows[i][j] 
                       for j in range(len(self.rows[0]))]
                       for i in range(len(self.rows))])

    def mat_mult(self, other):
        """Returns a new matrix that is the matrix product of this matrix and other"""
        if not isinstance(other, Matrix):
            raise TypeError("Matrix multiplication is only defined between matrices")
        m_rows, m_cols = self.shape()
        n_rows, n_cols = other.shape()
        if m_cols != n_rows:
            raise ValueError(f"Matrix dimensions incompatible for multiplication: {self.shape()} and {other.shape()}")
        
        result = [[sum(self.rows[i][k] * other.rows[k][j] 
                      for k in range(m_cols))
                  for j in range(n_cols)]
                  for i in range(m_rows)]
        return Matrix(result)

    def element_mult(self, other):
        """Returns a new matrix that is the element-wise product of this matrix and other"""
        if not isinstance(other, Matrix):
            raise TypeError("Element-wise multiplication is only defined between matrices")
        if self.shape() != other.shape():
            raise ValueError("Matrix dimensions must match for element-wise multiplication")
        return Matrix([[self.rows[i][j] * other.rows[i][j] 
                       for j in range(len(self.rows[0]))]
                       for i in range(len(self.rows))])

    def equals(self, other):
        """Returns True if this matrix equals other, False otherwise"""
        if not isinstance(other, Matrix):
            return False
        return self.rows == other.rows
    # Operator overloading methods
    def __mul__(self, other):
        """
        Implements multiplication operator (*).
        Supports both matrix multiplication (M * N) and scalar multiplication (M * c).
        
        """
        if isinstance(other, Matrix):
            return self.mat_mult(other)
        try:
            return self.scalarmul(float(other))
        except (TypeError, ValueError):
            raise TypeError("Matrix can only be multiplied by another matrix or a number")

    def __rmul__(self, other):
        """
        Implements reverse multiplication operator (c * M).
        
        """
        try:
            return self.scalarmul(float(other))
        except (TypeError, ValueError):
            raise TypeError("Matrix can only be multiplied by a number from the left")

    def __add__(self, other):
        """
        Implements addition operator (+).
        
        """
        return self.add(other)

    def __sub__(self, other):
        """
        Implements subtraction operator (-).
        
        """
        return self.sub(other)

In [62]:
def demonstrate_matrix_properties():
    
    # 2x2 matrices and identity matrix
    print("Creating test matrices:")
    A = Matrix([[1, 2], [3, 4]])
    B = Matrix([[5, 6], [7, 8]])
    C = Matrix([[9, 10], [11, 12]])
    I = Matrix([[1, 0], [0, 1]])  # Identity matrix
    
    print("Matrix A:")
    print(A)
    print("\nMatrix B:")
    print(B)
    print("\nMatrix C:")
    print(C)
    print("\nIdentity Matrix I:")
    print(I)
    print("\n")
    
    # 1.(AB)C = A(BC)
    print("1. Testing (AB)C = A(BC)")
    left_side = (A * B) * C
    right_side = A * (B * C)
    print("(AB)C:")
    print(left_side)
    print("\nA(BC):")
    print(right_side)
    print(f"Property holds: {left_side == right_side}\n")
    
    # 2. A(B + C) = AB + AC
    print("2. Testing A(B + C) = AB + AC")
    left_side = A * (B + C)
    right_side = (A * B) + (A * C)
    print("A(B + C):")
    print(left_side)
    print("\nAB + AC:")
    print(right_side)
    print(f"Property holds: {left_side == right_side}\n")
    
    # 3. AB ≠ BA
    print("3. Testing AB ≠ BA")
    ab = A * B
    ba = B * A
    print("AB:")
    print(ab)
    print("\nBA:")
    print(ba)
    print(f"AB = BA: {ab == ba} \n") #false
    
    # 4. AI = A
    print("4. Testing AI = A")
    ai = A * I
    print("AI:")
    print(ai)
    print("\nA:")
    print(A)
    print(f"Property holds: {ai == A}")


demonstrate_matrix_properties()

Creating test matrices:
Matrix A:
1 2
3 4

Matrix B:
5 6
7 8

Matrix C:
9 10
11 12

Identity Matrix I:
1 0
0 1


1. Testing (AB)C = A(BC)
(AB)C:
413 454
937 1030

A(BC):
413 454
937 1030
Property holds: True

2. Testing A(B + C) = AB + AC
A(B + C):
50 56
114 128

AB + AC:
50 56
114 128
Property holds: True

3. Testing AB ≠ BA
AB:
19 22
43 50

BA:
23 34
31 46
AB = BA: False 

4. Testing AI = A
AI:
1 2
3 4

A:
1 2
3 4
Property holds: True
